In [18]:
import pandas as pd
import numpy as np

In [19]:
questions = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/VLAT Questions.csv")
questions.head()

,id,dropped,vis,item,question:,option:,correct
0,1,no,VLAT_a,a_1,What was the price of a barrel of oil in Febru...,$57.36; $47.82; $50.24; $39.72,$50.24
1,2,no,VLAT_a,a_2,In which month was the price of a barrel of oi...,March; May; July; December,December
2,3,no,VLAT_a,a_3,What was the price range of a barrel of oil in...,$35 - $65; $48.36 - $60.95; $37.04 - $48.36; $...,$37.04 - $60.95
3,4,no,VLAT_a,a_4,"Over the course of the second half of 2015, th...",rising; falling; staying,falling
4,5,no,VLAT_a,a_5,About how much did the price of a barrel of oi...,$4; $15; $17; $45,$15


In [20]:
dropped_questions = set(questions[questions['dropped'] == "yes"]['id'].to_numpy())
dropped_questions

{13, 24, 26, 30, 39, 43, 50, 58}

In [21]:
questions['num_options'] = questions['option:'].apply(lambda x: len(x.split(";")))
questions.tail(3)

,id,dropped,vis,item,question:,option:,correct,num_options
58,59,no,VLAT_l,l_2,For which website was the number of unique vis...,Facebook; Amazon; Bing; Google,Google,4
59,60,no,VLAT_l,l_3,The number of unique visitors for Amazon was m...,True; False,FALSE,2
60,61,no,VLAT_l,l_4,Samsung is nested in the Financial category.,True; False,FALSE,2


In [22]:
questions = questions.set_index("id")

In [23]:
questions_meta = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/VLAT Questions Metadata.csv")
questions_meta = questions_meta.set_index("id").fillna("")
questions_meta.tail(3)

,vis,task,subtask,difficulty,discrimination
id,,,,,
59,treemap,find extremum,relative value,moderate,high
60,treemap,make comparisons,relative value,hard,high
61,treemap,identify hierarchical structure,,easy,medium


In [24]:
results_1 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/1_GPT 4.o_Final Results with Explanation/GPT_VLAT_1743188849.csv")
results_2 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/1_GPT 4.o_Final Results with Explanation/GPT_VLAT_1743188850.csv")
results_3 = pd.read_csv("C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/1_GPT 4.o_Final Results with Explanation/GPT_VLAT_1743188851.csv")

In [25]:
results = results_1.merge(results_2, on='id', suffixes=("", "_2"))
results = results.merge(results_3, on='id', suffixes=("", "_3"))
results = results[~results['id'].isin(dropped_questions)]
results = results.set_index("id")

In [26]:
len(results)

53

In [27]:
results.head()

,response,time,correct_bool,response_2,time_2,correct_bool_2,response_3,time_3,correct_bool_3
id,,,,,,,,,
1,$50.24,13.679759,True,$50.24,13.679759,True,$50.24,13.679759,True
2,December,11.586299,True,December,11.586299,True,December,11.586299,True
3,$37.04 - $60.95,14.837307,True,$37.04 - $60.95,14.837307,True,$37.04 - $60.95,14.837307,True
4,falling,12.526849,True,falling,12.526849,True,falling,12.526849,True
5,$11,9.743206,False,$11,9.743206,False,$11,9.743206,False


In [28]:
responses = results[['response', 'response_2', 'response_3']]
responses.head()

,response,response_2,response_3
id,,,
1,$50.24,$50.24,$50.24
2,December,December,December
3,$37.04 - $60.95,$37.04 - $60.95,$37.04 - $60.95
4,falling,falling,falling
5,$11,$11,$11


In [29]:
agreements = np.logical_and(responses['response'] == responses['response_2'], responses['response_2'] == responses['response_3'])
responses[~agreements]

,response,response_2,response_3
id,,,
27,53.9 kg,197.1 cm,53.9 kg
36,Sep-14,14-Sep,14-Sep
47,550 km,550 km,530 km


In [30]:
# Correct way to combine multiple logical conditions
omissions = np.logical_or(
    np.logical_or(responses['response'] == "Omit", responses['response_2'] == "Omit"),
    responses['response_3'] == "Omit"
)

# Then filter
responses[omissions]

,response,response_2,response_3
id,,,


In [31]:
scoring = results.copy()
scoring["correct_bool"] = np.where(scoring["response"] == "Omit", "Omit", scoring["correct_bool"])
scoring["correct_bool_2"] = np.where(scoring["response_2"] == "Omit", "Omit", scoring["correct_bool_2"])
scoring["correct_bool_3"] = np.where(scoring["response_3"] == "Omit", "Omit", scoring["correct_bool_3"])
scoring.head(6)

,response,time,correct_bool,response_2,time_2,correct_bool_2,response_3,time_3,correct_bool_3
id,,,,,,,,,
1,$50.24,13.679759,True,$50.24,13.679759,True,$50.24,13.679759,True
2,December,11.586299,True,December,11.586299,True,December,11.586299,True
3,$37.04 - $60.95,14.837307,True,$37.04 - $60.95,14.837307,True,$37.04 - $60.95,14.837307,True
4,falling,12.526849,True,falling,12.526849,True,falling,12.526849,True
5,$11,9.743206,False,$11,9.743206,False,$11,9.743206,False
6,14 Mbps,11.457596,False,14 Mbps,11.457596,False,14 Mbps,11.457596,False


In [32]:
def getScore(correct_arr):
    score = 0
    for i, item in enumerate(correct_arr):
        if item == "True":
            score += 1
        elif item == "False":
            score -= 1/(questions.loc[i,"num_options"] - 1)
    return score

In [33]:
for col in ["correct_bool", "correct_bool_2", "correct_bool_3"]:
    score = sum(scoring[col] == "True")
    print(col+":", round(score, 2), "(raw)")

correct_bool: 34 (raw)
correct_bool_2: 35 (raw)
correct_bool_3: 35 (raw)


In [34]:
for col in ["correct_bool", "correct_bool_2", "correct_bool_3"]:
    score = getScore(scoring[col].to_numpy())
    print(col+":", round(score, 2))

correct_bool: 24.0
correct_bool_2: 25.33
correct_bool_3: 25.33


In [35]:
# Chain logical_and operations correctly
scoring["correct_bool_consensus"] = np.where(
    # First logical_and chain for the condition
    np.logical_and(
        np.logical_and(scoring["response"] == "Omit", scoring["response_2"] == "Omit"),
        scoring["response_3"] == "Omit"
    ), 
    "Omit",
    # Second logical_and chain for the value
    np.logical_and(
        np.logical_and(scoring["correct_bool"] == "True", scoring["correct_bool_2"] == "True"),
        scoring["correct_bool_3"] == "True"
    )
)

# View the results
scoring[["correct_bool", "correct_bool_2", "correct_bool_3", "correct_bool_consensus"]].head(10)

,correct_bool,correct_bool_2,correct_bool_3,correct_bool_consensus
id,,,,
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
5,False,False,False,False
6,False,False,False,False
7,True,True,True,True
8,True,True,True,True
9,True,True,True,True


In [19]:
# First convert string "True" to actual boolean values if needed
scoring["correct_bool_consensus"] = scoring["correct_bool_consensus"].astype(str) == "True"

# Now calculate the raw score (sum of True values)
consensus_score_raw = scoring["correct_bool_consensus"].sum()
print(round(consensus_score_raw, 2), "(raw)")

# If getScore is a custom function, make sure it works with boolean arrays
# If it's not defined, you might need to implement it
consensus_score = getScore(scoring["correct_bool_consensus"].to_numpy())
print(round(consensus_score, 2))

38 (raw)
0


In [20]:
correct_bool_consensus = scoring[["correct_bool_consensus"]]
questions_meta_concensus_ans = correct_bool_consensus.join(questions_meta)
questions_meta_concensus_ans.head(5)

,correct_bool_consensus,vis,task,subtask,difficulty,discrimination
id,,,,,,
1,True,line chart,retrieve value,,easy,low
2,True,line chart,find extremum,,easy,low
3,False,line chart,determine range,,moderate,high
4,True,line chart,find correlations/trends,,easy,low
5,True,line chart,make comparisons,,moderate,high


In [21]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "True"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [22]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "False"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [23]:
for col in ['vis', 'task', 'difficulty']:
    vals = questions_meta_concensus_ans[questions_meta_concensus_ans['correct_bool_consensus'] == "Omit"][col].value_counts()
    print(vals, "\n")

Series([], Name: vis, dtype: int64) 

Series([], Name: task, dtype: int64) 

Series([], Name: difficulty, dtype: int64) 



In [24]:
questions_meta_concensus_ans[["vis", "correct_bool_consensus"]].value_counts(sort=False)

vis                     correct_bool_consensus
100% stacked bar chart  True                      3
area chart              False                     2
                        True                      2
bar chart               False                     1
                        True                      3
bubble chart            False                     2
                        True                      5
choropleth map          False                     2
                        True                      1
histogram               False                     1
                        True                      2
line chart              False                     1
                        True                      4
pie chart               True                      3
scatterplot             False                     2
                        True                      5
stacked area chart      True                      6
stacked bar chart       False                     4
                 

In [25]:
questions_meta_concensus_ans[["task", "correct_bool_consensus"]].value_counts(sort=False)

task                             correct_bool_consensus
determine range                  False                      3
                                 True                       2
find anomalies                   False                      2
find clusters                    True                       2
find correlations/trends         True                       5
find extremum                    False                      2
                                 True                      10
identify hierarchical structure  True                       1
make comparisons                 False                      3
                                 True                      10
retrieve value                   False                      5
                                 True                       8
dtype: int64

In [26]:
questions_meta_concensus_ans[["difficulty", "correct_bool_consensus"]].value_counts(sort=False)

difficulty  correct_bool_consensus
easy        False                      1
            True                      16
hard        False                      7
            True                      10
moderate    False                      7
            True                      12
dtype: int64

In [27]:
# Correct, hard
questions_meta_concensus_ans[np.logical_and(questions_meta_concensus_ans['correct_bool_consensus'] == "True",
                                            questions_meta_concensus_ans['difficulty'] == "hard")]

,correct_bool_consensus,vis,task,subtask,difficulty,discrimination
id,,,,,,
